In [31]:
import h5py
import glob
import scipy.io
from scipy.stats import stats
from sklearn.metrics import confusion_matrix
from sklearn.svm import NuSVC
import numpy as np
import random
from settings import *
# path = '/data/HBN/test/fmriprep_output/fmriprep/PythonData/'

In [32]:
from brainiak import isfc

In [91]:
subord = glob.glob(h5path+'sub*.h5')
with h5py.File(h5path+'ISC.h5') as hf:
    string_dt = h5py.special_dtype(vlen=str)
    hf.create_dataset("subord",data=np.array(subord).astype('|S70'),dtype=string_dt)
for task in ['DM','TP']:
    data = []
    for sub in subord:
        f = h5py.File(sub, 'r')
        D = {}
        for hem in ['L','R']:
            D[hem] = f[task][hem][:]
        data.append(np.concatenate(
            [D['L'], D['R']], axis=0))
    data = np.dstack(data)
    ISC = isfc.isc(data,collapse_subj=True)
    ISC_persubj = isfc.isc(data,collapse_subj=False)
    with h5py.File(h5path+'ISC.h5') as hf:
        grp = hf.create_group(task)
        grp.create_dataset('ISC', data=ISC)
        grp.create_dataset('ISC_persubj', data=ISC_persubj)

RuntimeError: Unable to create link (name already exists)

In [92]:
%matplotlib widget
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
# pick which random coords to display:
f = h5py.File('/data/Schema/intact/fsaverage6_adj.h5', 'r')
dispcoordsLH = f['lhinflatedcoords'][:]
randcoords = random.sample(range(len(dispcoordsLH[1])), 5000)
dispcoordsLH = dispcoordsLH[:,randcoords]
dispISCLH = ISC[:len(ISC)//2][randcoords] # For LH
dispcoordsRH = f['rhinflatedcoords'][:]
dispcoordsRH = dispcoordsRH[:,randcoords]
dispISCRH = ISC[len(ISC)//2:][randcoords] # For LH

f = h5py.File(h5path+'ISC.h5', 'r')
for idx,task in enumerate(['DM','TP']):
    ISC = f[task]['ISC'][:]
    ISC_persubj = f[task]['ISC_persubj'][:]
    ISC=np.nan_to_num(ISC)
    # For SUMA visualization:
    np.savetxt(path+task+'ISC.txt',ISC)
    np.savetxt(path+task+'ISC_LH.txt',ISC[:len(ISC)//2])
    np.savetxt(path+task+'ISC_RH.txt',ISC[len(ISC)//2:])
    dispISCLH = ISC[:len(ISC)//2][randcoords] # For LH
    dispISCRH = ISC[len(ISC)//2:][randcoords] # For LH
    fig = plt.subplot(2,3,idx)
    ax = fig.gca(projection='3d')
    x,y,z = dispcoordsRH
    brain = ax.scatter3D(x,y,z,c=dispISCRH, cmap='bwr');
    plt.colorbar(brain)
    fig = plt.subplot(2,3,idx+2)
    ax = fig.gca(projection='3d')
    x,y,z = dispcoordsLH
    brain = ax.scatter3D(x,y,z,c=dispISCLH, cmap='bwr');
    plt.colorbar(brain)

SyntaxError: invalid syntax (<ipython-input-92-2bfa7e073fe9>, line 25)

In [6]:
%matplotlib widget
fig = plt.figure()
#ax = plt.axes(projection='3d')
ax = fig.gca(projection='3d')

x,y,z = dispcoordsRH
brain = ax.scatter3D(x,y,z,c=dispISCRH, cmap='bwr');
plt.colorbar(brain)

FigureCanvasNbAgg()